# Middleware in LangChain 
mechanism that allows you to intercept, modify, or extend the behavior of chains and tools as they process inputs and outputs. It acts as a layer between the user and the core logic, enabling tasks like logging, validation, or custom processing without changing the main workflow.

## Example
Suppose you want to check if a user’s input contains any banned words before processing it in LangChain. You can use middleware to scan the input for banned words. If any are found, the middleware can block the request or return a warning, otherwise it lets the input continue to the main logic. This helps keep your application safe and clean without changing the core processing code.

In [3]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model('groq:qwen/qwen3-32b')
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000023178DCAFD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023178DCB9D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Summarization MiddleWare


In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import SystemMessage, HumanMessage

agent = create_agent(
    model='groq:qwen/qwen3-32b',
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model='groq:qwen/qwen3-32b',
            trigger = ("messages",10),
            keep = ("messages",4)
        )
    ]
)

In [5]:
### Run with thread ID
config={"configurable":{"thread_id":"test_thread"}}

In [6]:
question = [
    "what is 2+2",
    "what is 3+3",
    "what is 4+4",
    "what is 5+5",
    "what is 6+6",
    "what is 7+7",
    "what is 8+8",
    # "what is 9+9",
]
for q in question:
    response = agent.invoke({"messages" : [HumanMessage(content=q)]},config)
    print(f"Messages: {response}")
    print(f"Number of messages: {len(response['messages'])}")

Messages: {'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='2aa8111a-39e8-49ad-8868-79c87e029c42'), AIMessage(content='<think>\nOkay, the user is asking "what is 2+2". That\'s a basic arithmetic question. Let me start by recalling that addition is one of the fundamental operations in mathematics. When you add two numbers together, you\'re combining their quantities. So 2 plus 2 would be 4. But wait, maybe I should double-check that. Let me count: 2 objects plus another 2 objects make a total of 4 objects. Yes, that seems right.\n\nI wonder if there\'s any context where 2+2 could mean something else. In most everyday situations, it\'s straightforward addition. But in some specialized fields, like modular arithmetic or different number bases, the result might vary. For example, in base 3, 2+2 would be 11 (which is 4 in decimal). However, unless specified otherwise, the default assumption is base 10. Since the question doesn\'t mention any s